In [1]:
import pandas as pd


In [2]:

# Load the Excel file
file_path = "Slicer Extensions Download stats as of 2025.03.08.xlsx"  # Update path as needed
excel_data = pd.ExcelFile(file_path)

# Reshape all sheets
reshaped_data = []

for sheet in excel_data.sheet_names:
    # Parse sheet using second row as header (row index 1)
    df = excel_data.parse(sheet, header=1)
    
    # Rename columns for clarity
    df.rename(columns={df.columns[0]: "ID", df.columns[1]: "Extension"}, inplace=True)
    
    # Drop rows without an extension name
    df = df.dropna(subset=["Extension"])
    
    # Add sheet name as date info
    df["Date"] = sheet

    # Reshape the data: each row will be Extension, Version_Info, Downloads
    df_melted = df.melt(id_vars=["Extension", "Date"], 
                        var_name="Version_Info", 
                        value_name="Downloads")

    reshaped_data.append(df_melted)

# Combine all dataframes
long_df = pd.concat(reshaped_data, ignore_index=True)

# Clean up non-numeric download counts
long_df = long_df[pd.to_numeric(long_df["Downloads"], errors="coerce").notnull()]
long_df["Downloads"] = long_df["Downloads"].astype(int)

# Optional: remove rows with invalid version names (e.g., 'ID', 'post-5.x.x')
long_df = long_df[~long_df["Version_Info"].str.contains("post-|ID", na=False)]




In [3]:

# Group by Extension and Version_Info to get total downloads
grouped = long_df.groupby(["Extension", "Version_Info"]).agg(
    Total_Downloads=("Downloads", "sum")
).reset_index()

# Rank by total downloads
grouped["Rank"] = grouped.groupby("Version_Info")["Total_Downloads"].rank(
    method="dense", ascending=False
)

# Sort results
grouped_sorted = grouped.sort_values(by=["Version_Info", "Rank"])

# Save results if needed
# grouped_sorted.to_csv("slicer_extension_rankings.csv", index=False)

# Display top N (e.g., 10) entries for a quick look
print(grouped_sorted.head(10))


    Extension Version_Info  Total_Downloads  Rank
0           0        4.0.0                0   1.0
24          6        4.0.0                0   1.0
48          8        4.0.0                0   1.0
72          9        4.0.0                0   1.0
96         16        4.0.0                0   1.0
120        21        4.0.0                0   1.0
144        22        4.0.0                0   1.0
168        24        4.0.0                0   1.0
192        25        4.0.0                0   1.0
216        26        4.0.0                0   1.0
